# BPE Training

> Implementation of the `train_bpe` function for Byte-Pair Encoding tokenizer training.

In [ ]:
#| default_exp bpe_training

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
#| export
import regex as re
import json
import base64
from collections import Counter, defaultdict
from multiprocessing import Pool, cpu_count
from functools import partial
from typing import List, Dict, Tuple, Any, Pattern, Set
import os
import tempfile # For example usage/test block
import time # For example usage/test block

In [3]:
#| export
# GPT-2 pre-tokenization regex from the technical specification
PAT_REGEX: Pattern = re.compile(r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

In [4]:
#| export
def _process_segment(segment_text: str, pat_regex: Pattern) -> List[List[int]]:
    """Internal helper function to pre-tokenize a text segment and convert to UTF-8 byte lists.

    Args:
        segment_text: A string segment of the corpus.
        pat_regex: The pre-compiled regex pattern for pre-tokenization.

    Returns:
        A list where each inner list contains the UTF-8 byte values (integers 0-255)
        for a pre-token found in the segment_text.
    """
    pre_tokens_as_int_lists: List[List[int]] = []
    if not segment_text: # Handle empty segments gracefully
        return []
    for match in pat_regex.finditer(segment_text):
        pre_token_str = match.group(0)
        pre_tokens_as_int_lists.append(list(pre_token_str.encode('utf-8')))
    return pre_tokens_as_int_lists

In [5]:
#| hide
def _replace_pair_in_sequence(
    sequence: Tuple[int, ...],
    pair_to_replace: Tuple[int, int],
    new_id: int
) -> Tuple[int, ...]:
    """Replaces all occurrences of a pair of token IDs with a new token ID in a sequence.

    Args:
        sequence: The tuple of token IDs representing a word/token.
        pair_to_replace: A tuple of two token IDs (e.g., (id1, id2)) to be replaced.
        new_id: The new token ID that will replace the pair.

    Returns:
        A new tuple of token IDs with all occurrences of the pair replaced.
    """
    new_sequence_list: List[int] = []
    i = 0
    while i < len(sequence):
        # Check if the current position and the next form the pair to replace
        if i + 1 < len(sequence) and \
           sequence[i] == pair_to_replace[0] and \
           sequence[i+1] == pair_to_replace[1]:
            new_sequence_list.append(new_id)
            i += 2 # Move past the pair
        else:
            new_sequence_list.append(sequence[i])
            i += 1 # Move to the next token
    return tuple(new_sequence_list)

In [6]:
#| export
def train_bpe(
    input_path: str,         # Path to the raw text file
    vocab_size: int,         # Desired final vocabulary size
    special_tokens: List[str] # List of special token strings
) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:
    """Trains a Byte-Pair Encoding (BPE) tokenizer from a raw text file.

    Args:
        input_path: Path to the input text file (UTF-8 encoded).
        vocab_size: The target size of the vocabulary to be learned.
        special_tokens: A list of strings representing special tokens (e.g., '<|endoftext|>').

    Returns:
        A tuple containing:
        - vocab (Dict[int, bytes]): The learned vocabulary mapping token IDs (int) to byte sequences (bytes).
        - merges (List[Tuple[bytes, bytes]]): The list of learned BPE merge rules, where each rule
          is a tuple of two byte sequences that are merged.

    Raises:
        AssertionError: If vocab_size is too small to accommodate base bytes and special tokens.
        FileNotFoundError: If input_path does not exist.
    """
    # nbdev: CWD to project root for pathing (this is a comment, nbdev handles it for notebook execution)
    
    # Calculate the number of unique byte sequences from special tokens
    unique_special_token_byte_values: Set[bytes] = set()
    if special_tokens:
        for st_str in special_tokens:
            unique_special_token_byte_values.add(st_str.encode("utf-8"))
    
    min_required_vocab_size_for_assertion = 256 # Start with base bytes
    temp_base_bytes_for_assertion = {bytes([i]) for i in range(256)}
    for st_byte_val in unique_special_token_byte_values:
        if st_byte_val not in temp_base_bytes_for_assertion:
            min_required_vocab_size_for_assertion +=1

    assert vocab_size >= min_required_vocab_size_for_assertion, \
        f"vocab_size ({vocab_size}) must be at least {min_required_vocab_size_for_assertion} to accommodate all unique special tokens and 256 base bytes."

    vocab: Dict[int, bytes] = {}
    current_id: int = 0
    merges: List[Tuple[bytes, bytes]] = [] 

    # 1. Vocabulary Initialization (Step D.2)
    _temp_seen_special_bytes: Set[bytes] = set()
    temp_special_vocab: Dict[int, bytes] = {} 
    temp_special_id_counter: int = 0
    
    if special_tokens:
        unique_st_strings = sorted(list(set(special_tokens)))
        for st_str in unique_st_strings:
            st_bytes = st_str.encode("utf-8")
            if st_bytes not in _temp_seen_special_bytes:
                temp_special_vocab[temp_special_id_counter] = st_bytes
                _temp_seen_special_bytes.add(st_bytes)
                temp_special_id_counter += 1
    
    for _temp_id, byte_val in sorted(temp_special_vocab.items()): 
        vocab[current_id] = byte_val
        current_id += 1
        
    current_vocab_byte_values = set(vocab.values()) 
    for i in range(256):
        byte_val = bytes([i])
        if byte_val not in current_vocab_byte_values:
            vocab[current_id] = byte_val
            current_id += 1
            current_vocab_byte_values.add(byte_val) 
            
    initial_vocab_size: int = len(vocab)
    if vocab_size < initial_vocab_size:
        print(f"Warning: Requested vocab_size {vocab_size} is less than the initial vocab size {initial_vocab_size} \n"
              f"(derived from special tokens and 256 base bytes). \n"
              f"Returning the initial vocabulary of size {initial_vocab_size}. No merges will be performed.")
        return vocab, merges

    # 2. Corpus Processing and Parallel Pre-tokenization (Step D.3)
    try:
        with open(input_path, 'r', encoding='utf-8') as f:
            corpus_text = f.read()
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_path}")
        raise
    except Exception as e:
        print(f"Error reading input file {input_path}: {e}")
        raise

    text_segments: List[str]
    if special_tokens:
        escaped_special_tokens = [re.escape(st) for st in sorted(list(set(special_tokens)), key=len, reverse=True)]
        delimiter_pattern_str = "|".join(escaped_special_tokens)
        delimiter_pattern = re.compile(delimiter_pattern_str)
        text_segments = delimiter_pattern.split(corpus_text)
    else:
        text_segments = [corpus_text]
    
    all_pre_tokens_int_lists: List[List[int]] = []
    num_processes = 1
    if text_segments and any(s.strip() for s in text_segments):
        try:
            non_empty_segments = [s for s in text_segments if s.strip()] 
            if non_empty_segments:
                num_processes = min(cpu_count(), len(non_empty_segments))
                if num_processes == 0: num_processes = 1 
                
                with Pool(processes=num_processes) as pool:
                    map_func = partial(_process_segment, pat_regex=PAT_REGEX)
                    try:
                        results_list_of_lists = pool.map(map_func, non_empty_segments)
                        for sublist in results_list_of_lists:
                            all_pre_tokens_int_lists.extend(sublist)
                    except Exception as e:
                        print(f"Error during parallel pre-tokenization: {e}")
                        pass 
        except NotImplementedError: 
            num_processes = 1 
            print("Warning: cpu_count() not implemented. Defaulting to 1 process for pre-tokenization.")
            if not all_pre_tokens_int_lists: 
                for segment_text in text_segments:
                    if segment_text.strip(): 
                        all_pre_tokens_int_lists.extend(_process_segment(segment_text, PAT_REGEX))
        except Exception as e: 
            print(f"An unexpected error occurred with multiprocessing.Pool: {e}. Defaulting to sequential pre-tokenization.")
            num_processes = 1
            if not all_pre_tokens_int_lists: 
                for segment_text in text_segments:
                    if segment_text.strip():
                        all_pre_tokens_int_lists.extend(_process_segment(segment_text, PAT_REGEX))
    
    if not all_pre_tokens_int_lists and vocab_size > initial_vocab_size:
         print(f"Warning: No tokens found in corpus '{input_path}' after pre-tokenization. \n"
               f"Returning initial vocab of size {initial_vocab_size}. Cannot reach target vocab_size {vocab_size}.")
         return vocab, merges 

    # 3. Word Counts (Step D.4)
    word_counts = Counter(tuple(token_as_int_list) for token_as_int_list in all_pre_tokens_int_lists if token_as_int_list)
    
    # 4. BPE Merge Loop (Step D.6)
    num_merges_needed = vocab_size - len(vocab)
    byte_map: Dict[int, bytes] = vocab.copy() 

    for i in range(num_merges_needed):
        if not word_counts:
            print(f"Warning: Word counts became empty. Stopping BPE training early at vocab size {len(vocab)} after {i} merges.")
            break

        pair_freqs = Counter()
        for token_tuple_ints, count in word_counts.items():
            for j in range(len(token_tuple_ints) - 1):
                pair = (token_tuple_ints[j], token_tuple_ints[j+1])
                pair_freqs[pair] += count
        
        if not pair_freqs:
            print(f"Warning: No more pairs to merge. Stopping BPE training early at vocab size {len(vocab)} after {i} merges.")
            break

        # Find Best Pair
        max_freq = max(pair_freqs.values())
        candidate_pairs = [p for p, freq in pair_freqs.items() if freq == max_freq]
        
        best_pair_int_tuple = candidate_pairs[0]
        if len(candidate_pairs) > 1:
            max_byte_tuple_representation = (byte_map[best_pair_int_tuple[0]], byte_map[best_pair_int_tuple[1]])
            for k_idx in range(1, len(candidate_pairs)):
                p_int_candidate = candidate_pairs[k_idx]
                current_byte_tuple_representation = (byte_map[p_int_candidate[0]], byte_map[p_int_candidate[1]])
                if current_byte_tuple_representation > max_byte_tuple_representation:
                    max_byte_tuple_representation = current_byte_tuple_representation
                    best_pair_int_tuple = p_int_candidate
            
        # Perform Merge
        new_token_id = current_id
        
        p1_bytes = byte_map[best_pair_int_tuple[0]]
        p2_bytes = byte_map[best_pair_int_tuple[1]]
        merged_bytes = p1_bytes + p2_bytes

        vocab[new_token_id] = merged_bytes
        byte_map[new_token_id] = merged_bytes 
        merges.append((p1_bytes, p2_bytes)) 
        current_id += 1

        # Update word_counts
        new_word_counts = Counter()
        for token_tuple_ints, count in word_counts.items():
            updated_token_tuple_ints = _replace_pair_in_sequence(token_tuple_ints, best_pair_int_tuple, new_token_id)
            new_word_counts[updated_token_tuple_ints] += count
        word_counts = new_word_counts
            
        if len(vocab) >= vocab_size:
            break
        
    return vocab, merges

In [7]:
#| hide
# Example usage/test block for train_bpe, now including the merge loop.
if __name__ == '__main__':
    # --- Tests for _replace_pair_in_sequence ---
    seq1 = (1, 2, 3, 1, 2, 4, 1, 2); pair1 = (1, 2); new_id1 = 99; expected1 = (99, 3, 99, 4, 99)
    assert _replace_pair_in_sequence(seq1, pair1, new_id1) == expected1, "_replace_pair_in_sequence test 1 FAILED"
    print("_replace_pair_in_sequence test 1 PASSED.")

    # --- Full train_bpe test with a small example ---
    temp_file_path_full_test = ""
    try:
        with tempfile.NamedTemporaryFile(mode='w', delete=False, encoding='utf-8') as tmp_file:
            tmp_file.write("abab aa ab\n") 
            tmp_file.write("abab cc ab\n") 
            temp_file_path_full_test = tmp_file.name
        
        print(f"\nTesting full train_bpe with merge loop on: {temp_file_path_full_test}")
        test_vocab_size = 258 
        test_special_tokens = []

        start_time = time.time()
        learned_vocab, learned_merges = train_bpe(temp_file_path_full_test, test_vocab_size, test_special_tokens)
        end_time = time.time()

        print(f"train_bpe (full test) took {end_time - start_time:.4f} seconds.")
        print(f"Learned vocab size: {len(learned_vocab)}")
        print(f"Number of merges: {len(learned_merges)}")

        print("Learned merged tokens (IDs >= 256):")
        for token_id, token_bytes in learned_vocab.items():
            if token_id >= 256:
                print(f"  ID {token_id}: {token_bytes!r} (decoded: '{token_bytes.decode('utf-8', 'replace')}')")
        
        print("Learned merges:")
        for m_idx, (p1, p2) in enumerate(learned_merges):
            print(f"  Merge {m_idx+1}: ({p1!r}, {p2!r}) -> (decoded: ('{p1.decode('utf-8', 'replace')}', '{p2.decode('utf-8', 'replace')}'))")

        assert len(learned_vocab) <= test_vocab_size, "Vocab size exceeds target"
        # initial vocab for no special tokens is 256.
        # num_merges_needed = 258 - 256 = 2
        if test_vocab_size > 256: # Only assert if merges were expected
            assert len(learned_merges) == test_vocab_size - 256, f"Number of merges is incorrect. Expected {test_vocab_size - 256}, Got {len(learned_merges)}"
        
        if learned_merges:
            assert learned_merges[0] == (b'a', b'b'), f"First merge unexpected: {learned_merges[0]}"
            print("First merge (a,b) is as expected.")
            if len(learned_merges) > 1:
                 assert learned_merges[1] == (b'ab', b'ab'), f"Second merge unexpected: {learned_merges[1]}"
                 print("Second merge (ab,ab) is as expected.")
        else:
            print("No merges learned, skipping specific merge assertions.")

    except Exception as e:
        print(f"An unexpected error occurred during full train_bpe test: {e}")
        import traceback
        traceback.print_exc()
    finally:
        if temp_file_path_full_test and os.path.exists(temp_file_path_full_test):
            os.remove(temp_file_path_full_test)
            print(f"Cleaned up temporary file: {temp_file_path_full_test}")
    
    temp_file_path_assert = ""
    try:
        with tempfile.NamedTemporaryFile(mode='w', delete=False, encoding='utf-8') as tmp_file:
            tmp_file.write("text")
            temp_file_path_assert = tmp_file.name
        print("\nTesting vocab_size assertion again:")
        train_bpe(temp_file_path_assert, 10, ["<|endoftext|>"])
        assert False, "AssertionError for small vocab_size not raised (re-test)"
    except AssertionError as e:
        print(f"Caught expected assertion for small vocab_size (re-test): {e}")
    finally:
        if temp_file_path_assert and os.path.exists(temp_file_path_assert):
            os.remove(temp_file_path_assert)

    print("All train_bpe notebook tests (including merge loop basics) completed.")


_replace_pair_in_sequence test 1 PASSED.

Testing full train_bpe with merge loop on: /var/folders/1_/__n8_ny14_g0tlyfd57xj1rw0000gn/T/tmp9sgrqr7f
train_bpe (full test) took 0.0130 seconds.
Learned vocab size: 258
Number of merges: 2
Learned merged tokens (IDs >= 256):
  ID 256: b'ab' (decoded: 'ab')
  ID 257: b'abab' (decoded: 'abab')
Learned merges:
  Merge 1: (b'a', b'b') -> (decoded: ('a', 'b'))
  Merge 2: (b'ab', b'ab') -> (decoded: ('ab', 'ab'))
First merge (a,b) is as expected.
Second merge (ab,ab) is as expected.
Cleaned up temporary file: /var/folders/1_/__n8_ny14_g0tlyfd57xj1rw0000gn/T/tmp9sgrqr7f

Testing vocab_size assertion again:
Caught expected assertion for small vocab_size (re-test): vocab_size (10) must be at least 257 to accommodate all unique special tokens and 256 base bytes.
All train_bpe notebook tests (including merge loop basics) completed.
